In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='svg'

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [6]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


исправляем опечатки в годах, а вместо диапазона возьмем среднее

In [7]:
data.loc[data['HouseYear']==20052011, 'HouseYear'] = 2008
data.loc[data['HouseYear']==4968, 'HouseYear'] = 1968

In [8]:
data['HouseYear'].unique()

array([1969, 1978, 1968, 1977, 1976, 2011, 1960, 2014, 1973, 1959, 1999,
       1980, 1979, 1983, 2001, 2012, 2002, 1996, 1964, 2018, 1972, 1965,
       1984, 1961, 1971, 1963, 2017, 1970, 1981, 2003, 2016, 1991, 1975,
       2006, 2009, 1985, 1974, 1994, 2000, 1987, 1998, 2005, 1990, 1982,
       1997, 2015, 2008, 2010, 2004, 2007, 1967, 1957, 1962, 1993, 1966,
       1955, 1937, 1992, 1954, 1995, 2019, 1948, 1986, 2013, 1989, 1958,
       1938, 1956, 1988, 2020, 1951, 1952, 1935, 1914, 1932, 1950, 1917,
       1918, 1940, 1942, 1939, 1934, 1931, 1919, 1912, 1953, 1936, 1947,
       1929, 1930, 1933, 1941, 1916, 1910, 1928])

In [9]:
for field in data.columns:
    print(field, ': \t', data[field].nunique())

Id : 	 10000
DistrictId : 	 205
Rooms : 	 9
Square : 	 10000
LifeSquare : 	 7887
KitchenSquare : 	 58
Floor : 	 33
HouseFloor : 	 44
HouseYear : 	 95
Ecology_1 : 	 129
Ecology_2 : 	 2
Ecology_3 : 	 2
Social_1 : 	 51
Social_2 : 	 142
Social_3 : 	 30
Healthcare_1 : 	 79
Helthcare_2 : 	 7
Shops_1 : 	 16
Shops_2 : 	 2
Price : 	 10000


In [10]:
data['EcoBA'] = data['Ecology_3'] + data['Ecology_2']
data['EcoBA'].unique()

array(['BB', 'BA', 'AB'], dtype=object)

In [11]:
data.groupby(['EcoBA'], sort=True)[['Healthcare_1']].agg(['mean', 'median'])

Healthcare_1        
              mean  median
EcoBA                     
AB     1638.865455  1130.0
BA     1377.979381  1548.0
BB     1109.945549   900.0

Экологический класс в целом коррелирует с полем Healthcare_1:

In [12]:
data.groupby(['Ecology_2'], sort=True)[['Healthcare_1']].agg(['mean', 'median'])

Healthcare_1        
                  mean  median
Ecology_2                     
A          1377.979381  1548.0
B          1138.437806   900.0

In [13]:
data.groupby(['Ecology_3'], sort=True)[['Healthcare_1']].agg(['mean', 'median'])

Healthcare_1        
                  mean  median
Ecology_3                     
A          1638.865455  1130.0
B          1115.222448   900.0

In [14]:
data.groupby(['Ecology_2','Shops_2'], sort=True)[['Price', 'Rooms', 'Shops_1', 'Healthcare_1']].mean()

Price     Rooms   Shops_1  Healthcare_1
Ecology_2 Shops_2                                                 
A         A        159386.651546  1.642857  0.000000    370.000000
          B        198833.005156  1.951807  2.000000   1548.000000
B         A        192908.557092  1.895191  2.036991   1420.142857
          B        216256.619186  1.889903  4.453916   1130.494461

Категория магазинов: вероятно, shops_1 это количество близлежащих магазинов

In [15]:
data.groupby(['Shops_2'])[['Shops_1', 'Price']].mean()

,Shops_1,Price
Shops_2,,
A,2.002424,192339.700513
B,4.431717,216098.999571


Средние социальные показатели меньше в категории A чем у B (признак Shops_2), как и стоимость квартир

In [16]:
data.groupby(['Shops_2'])[['Social_1', 'Social_2', 'Social_3', 'Price']].agg(['mean', 'median'])

Social_1            Social_2         Social_3                 Price  \
              mean median         mean median      mean median           mean   
Shops_2                                                                         
A        16.230303      2  3431.099394    629  1.458182      1  192339.700513   
B        25.447411     25  5524.895586   5469  8.630954      2  216098.999571   

                        
                median  
Shops_2                 
A        181620.004130  
B        193446.129454

квартира имеет три разные площади:
- жилая площадь квартиры;
- площадь квартиры;
- общая площадь квартиры.

Жилая площадь – это сумма площадей жилых комнат. Общая площадь квартиры – это сумма площадей всех комнат, составляющих данную квартиру, в том числе подсобных помещений, кроме лоджий, балконов, веранд и террас. К подсобным помещениям относятся площади кухонь, коридоров, ванных, санузлов, встроенных шкафов, кладовых, а также площадь, занятая внутриквартирной лестницей. Ну и площадь квартиры – это сумма площадей всех помещений, составляющих квартиру, в том числе балконы, лоджии и террасы, площадь которых посчитана с определенным коэффициентом

In [17]:
data['DiffSquare'] = data['Square'] - data['LifeSquare'] - data['KitchenSquare']

In [18]:
data.loc[:, ['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'DiffSquare', 'Price']].head()

,Rooms,Square,LifeSquare,KitchenSquare,DiffSquare,Price
0,2.0,47.981561,29.442751,6.0,12.538811,184966.930730
1,3.0,65.683640,40.049543,8.0,17.634097,300009.450063
2,2.0,44.947953,29.197612,0.0,15.750341,220925.908524
3,2.0,53.352981,52.731512,9.0,-8.378531,175616.227217
4,1.0,39.649192,23.776169,7.0,8.873023,150226.531644


In [19]:
data.groupby(['Rooms'])[['Id']].count()

,Id
Rooms,
0.0,8
1.0,3705
2.0,3880
3.0,2235
4.0,150
5.0,18
6.0,1
10.0,2
19.0,1


In [20]:
data.loc[data['Rooms'] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,EcoBA,DiffSquare
1397,12638,27,0.0,138.427694,136.215499,0.0,4,3.0,2016,0.075424,...,11,3097,0,NaN,0,0,B,268394.744389,BB,2.212195
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,...,9,1892,0,NaN,0,1,B,302211.260887,BB,1.701236
2269,7317,27,0.0,41.790881,NaN,0.0,13,0.0,1977,0.211401,...,9,1892,0,NaN,0,1,B,98129.976788,BB,NaN
3911,770,28,0.0,49.483501,NaN,0.0,16,0.0,2015,0.118537,...,30,6207,1,1183.0,1,0,B,217009.338463,BB,NaN
4366,456,6,0.0,81.491446,NaN,0.0,4,0.0,1977,0.243205,...,5,1564,0,540.0,0,0,B,212864.799112,BB,NaN
4853,3224,27,0.0,2.377248,0.873147,0.0,1,0.0,1977,0.017647,...,2,469,0,NaN,0,0,B,126596.941798,BB,1.504101
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,...,43,8429,3,NaN,3,9,B,158998.110646,BB,10.351986
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,...,9,1892,0,NaN,0,1,B,219281.918007,BB,2.637145


Квартира с 0 комнат явно студия - судя по стоимости, отсутствия кухни и оставшейся площади на санузел

In [21]:
data.loc[data['Rooms'] == 0, ['Rooms']] = 1

In [22]:
data = data.loc[data['Square'] > 3]

Скорректируем площадь там, где разница DiffSquare получилась отрицательной

In [23]:
data.loc[data['DiffSquare'] < 0, ['Square']] = data['LifeSquare'] + data['KitchenSquare']
data.loc[data['DiffSquare'] < 0, ['DiffSquare']] = 0

In [24]:
data.groupby(['Rooms'])[['DiffSquare']].median()

,DiffSquare
Rooms,
1.0,9.024500
2.0,11.753679
3.0,16.714846
4.0,20.399553
5.0,21.222706
6.0,14.712089
10.0,12.642244
19.0,13.226758


In [25]:
data = data.loc[data['Rooms'] < 6]

In [26]:
mdfs = data.groupby(['Rooms'])[['DiffSquare']].median().reset_index().rename(columns={'DiffSquare': 'MedianDiff'})
data = pd.merge(data, mdfs, on=[
    'Rooms'], how='left')

Заполним пропуски жилой площади разностью между площадью и срейдней MedianDiff с таким же количеством комнат

In [27]:
data['LifeSquare'] = data['LifeSquare'].fillna(data['Square'] - data['MedianDiff'])
data['DiffSquare'] = data['Square'] - data['LifeSquare'] - data['KitchenSquare']
data.loc[data['DiffSquare'] < 0, ['DiffSquare']] = 0

In [28]:
data['SSQ'] = data['Square'] ** 2
data['LSSQ'] = data['LifeSquare'] ** 2
data['KSSQ'] = data['KitchenSquare'] ** 2
data['DSSQ'] = data['DiffSquare'] ** 2

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 9990
Data columns (total 27 columns):
Id               9991 non-null int64
DistrictId       9991 non-null int64
Rooms            9991 non-null float64
Square           9991 non-null float64
LifeSquare       9991 non-null float64
KitchenSquare    9991 non-null float64
Floor            9991 non-null int64
HouseFloor       9991 non-null float64
HouseYear        9991 non-null int64
Ecology_1        9991 non-null float64
Ecology_2        9991 non-null object
Ecology_3        9991 non-null object
Social_1         9991 non-null int64
Social_2         9991 non-null int64
Social_3         9991 non-null int64
Healthcare_1     5199 non-null float64
Helthcare_2      9991 non-null int64
Shops_1          9991 non-null int64
Shops_2          9991 non-null object
Price            9991 non-null float64
EcoBA            9991 non-null object
DiffSquare       9991 non-null float64
MedianDiff       9991 non-null float64
SSQ              9

In [30]:
data = data.loc[:, ['Id', 'DistrictId', 'Rooms', 'HouseYear',
       'Social_1', 'Social_2', 'Social_3', 'Shops_2', 'Price',
                    'Ecology_1', 'Ecology_2', 'Ecology_3',
                    'SSQ', 'LSSQ', 'KSSQ', 'DSSQ']]

In [31]:
data = pd.get_dummies(data)

In [32]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9991 entries, 0 to 9990
Data columns (total 19 columns):
Id             9991 non-null int64
DistrictId     9991 non-null int64
Rooms          9991 non-null float64
HouseYear      9991 non-null int64
Social_1       9991 non-null int64
Social_2       9991 non-null int64
Social_3       9991 non-null int64
Price          9991 non-null float64
Ecology_1      9991 non-null float64
SSQ            9991 non-null float64
LSSQ           9991 non-null float64
KSSQ           9991 non-null float64
DSSQ           9991 non-null float64
Shops_2_A      9991 non-null uint8
Shops_2_B      9991 non-null uint8
Ecology_2_A    9991 non-null uint8
Ecology_2_B    9991 non-null uint8
Ecology_3_A    9991 non-null uint8
Ecology_3_B    9991 non-null uint8
dtypes: float64(7), int64(6), uint8(6)
memory usage: 1.1 MB


In [33]:
train_fields = ['Rooms', 'HouseYear', 'Social_1', 'Social_2',
       'Social_3', 'Ecology_1', 'SSQ', 'LSSQ', 'KSSQ',
       'DSSQ', 'Shops_2_A', 'Shops_2_B', 'Ecology_2_A', 'Ecology_2_B',
       'Ecology_3_A', 'Ecology_3_B']

In [34]:
train, valid = train_test_split(data, test_size = 0.25, random_state=42)

In [35]:
rf = RandomForestRegressor(n_estimators=1001, max_depth=15, min_samples_leaf=2, random_state=42)
rf.fit(train.loc[:, train_fields], train['Price'])
pred = rf.predict(train.loc[:, train_fields])
print(r2(train['Price'], pred))
pred_valid = rf.predict(valid.loc[:, train_fields])
r2_valid = r2(valid['Price'], pred_valid)
print(r2_valid)

0.9124400008233364
0.7398567961208875


Предсказание на тестовом множестве

In [36]:
test = pd.read_csv('test.csv')

In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [38]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [39]:
test.groupby(['Rooms']).count()

,Id,DistrictId,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
Rooms,,,,,,,,,,,,,,,,,,
0.0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2
1.0,1769,1769,1769,1362,1769,1769,1769,1769,1769,1769,1769,1769,1769,1769,902,1769,1769,1769
2.0,2030,2030,2030,1587,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030,1078,2030,2030,2030
3.0,1099,1099,1099,915,1099,1099,1099,1099,1099,1099,1099,1099,1099,1099,593,1099,1099,1099
4.0,90,90,90,84,90,90,90,90,90,90,90,90,90,90,47,90,90,90
5.0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,1,7,7,7
6.0,2,2,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2
17.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [40]:
test.loc[test['Rooms']>5]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
2071,10793,23,6.0,110.750226,NaN,0.0,2,2.0,2015,0.014073,B,B,2,475,0,NaN,0,0,B
3217,4058,27,6.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,B,B,53,14892,4,NaN,1,4,B
3398,1435,111,17.0,52.866107,32.528342,8.0,15,17.0,1987,0.093443,B,B,23,4635,5,3300.0,2,4,B


In [41]:
test.loc[test['Rooms'] == 17, ['Rooms']] = 1

In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [43]:
test.loc[test['Rooms'] == 0, ['Rooms']] = 1

In [44]:
test['DiffSquare'] = test['Square'] - test['LifeSquare'] - test['KitchenSquare']

In [45]:
test.loc[test['DiffSquare'] < 0, ['Square']] = test['LifeSquare'] + test['KitchenSquare']

In [46]:
test.loc[test['DiffSquare'] < 0, ['DiffSquare']] = 0

In [47]:
mdfs_test = test.groupby(['Rooms'])[['DiffSquare']].median().reset_index().rename(columns={'DiffSquare': 'MedianDiff'})
test = pd.merge(test, mdfs_test, on=['Rooms'], how='left')
test['LifeSquare'] = test['LifeSquare'].fillna(test['Square'] - test['MedianDiff'])
test['DiffSquare'] = test['Square'] - test['LifeSquare'] - test['KitchenSquare']
test.loc[test['DiffSquare'] < 0, ['DiffSquare']] = 0
test['SSQ'] = test['Square'] ** 2
test['LSSQ'] = test['LifeSquare'] ** 2
test['KSSQ'] = test['KitchenSquare'] ** 2
test['DSSQ'] = test['DiffSquare'] ** 2

In [48]:
test = pd.get_dummies(test)

In [49]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 28 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
DiffSquare       5000 non-null float64
MedianDiff       5000 non-null float64
SSQ              5000 non-null float64
LSSQ             5000 non-null float64
KSSQ             5000 non-null float64
DSSQ             5000 non-null float64
Ecology_2_A      5000 non-null uint8
Ecology_2_B     

In [50]:
test = test.loc[:, ['Id', 'Rooms', 'HouseYear', 'Social_1', 'Social_2',
       'Social_3', 'Ecology_1', 'SSQ', 'LSSQ', 'KSSQ',
       'DSSQ', 'Shops_2_A', 'Shops_2_B', 'Ecology_2_A', 'Ecology_2_B',
       'Ecology_3_A', 'Ecology_3_B']]

In [51]:
pred_test = rf.predict(test.loc[:, train_fields])

In [52]:
pred_test.shape

(5000,)

In [53]:
test['Price'] = pred_test

In [54]:
test.head()

,Id,Rooms,HouseYear,Social_1,Social_2,Social_3,Ecology_1,SSQ,LSSQ,KSSQ,DSSQ,Shops_2_A,Shops_2_B,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Price
0,725,2.0,1972,11,2748,1,0.310199,2488.278112,1117.750892,36.0,109.199609,0,1,0,1,0,1,154941.978514
1,15856,2.0,1977,6,1437,3,0.075779,4797.388585,3318.383960,1.0,113.588006,0,1,0,1,0,1,204246.943316
2,5480,1.0,1909,30,7538,87,0.000000,781.104477,254.346563,144.0,0.000000,0,1,0,1,0,1,196156.056475
3,15664,2.0,2007,23,4583,3,0.101872,5335.807118,2697.851066,81.0,146.549600,0,1,0,1,0,1,303458.295125
4,14275,1.0,2017,2,629,1,0.072158,2258.826252,1882.481162,1.0,9.856721,1,0,0,1,0,1,134894.209230


In [55]:
test.loc[:, ['Id', 'Price']].to_csv('DSelunin_predictions.csv', index=None)